In [1]:
from photo_utils import *
import pandas as pd
import os

In [34]:
# image_list = os.listdir('flooded_imagery_CB_2023')
image_list = os.listdir('depth_model_data/flooded_imagery_CB_2023/daylight_images')

In [35]:
image_list[0]

'CAM_CB_01_20231217153626.jpg'

In [11]:
wtr_lvl_data = pd.read_csv('flood_tracking/flood_events.csv')

In [16]:
wtr_lvl_data.head()

,flood_event,sensor_ID,time_UTC,time_EST,water_level,start_time_UTC,end_time_UTC,start_time_EST,end_time_EST,duration_(hours),max_road_water_level_(ft),max_road_water_level_(m)
0,1,CB_02,2023-01-22 12:16:30+00:00,2023-01-22 07:16:30-05:00,0.083693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CB_02,2023-01-22 12:22:30+00:00,2023-01-22 07:22:30-05:00,0.156578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,CB_03,2023-01-22 12:23:49+00:00,2023-01-22 07:23:49-05:00,0.071578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,CB_02,2023-01-22 12:28:30+00:00,2023-01-22 07:28:30-05:00,0.208207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,CB_03,2023-01-22 12:29:49+00:00,2023-01-22 07:29:49-05:00,0.103121,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Convert utc_time column to datetime
wtr_lvl_data['time_UTC'] = pd.to_datetime(wtr_lvl_data['time_UTC'])

In [36]:
# List to store the results
results = []

# Iterate over image filenames
for filename in image_list:
    # Extract the sensor id and timestamp
    sensor_id = extract_sensor_name(filename)[4:]
    timestamp = extract_timestamp(filename)

    timestamp = pytz.utc.localize(datetime.strptime(timestamp, "%Y%m%d%H%M%S"))
    
    # Filter the dataframe by sensor id
    filtered_df = wtr_lvl_data[wtr_lvl_data['sensor_ID'] == sensor_id]
    
    # Find the closest timestamp
    closest_row = filtered_df.iloc[(filtered_df['time_UTC'] - timestamp).abs().argsort()[:1]]
    
    # Append the result
    if not closest_row.empty:
        result = {
            'image_filename': filename,
            'closest_utc_time': closest_row['time_UTC'].values[0],
            'water_level': closest_row['water_level'].values[0]
        }
        results.append(result)

# Convert the results to a dataframe
results_df = pd.DataFrame(results)

# Save the results to a CSV
results_df.to_csv('matched_image_data.csv', index=False)

In [32]:
copy_daylight_images('depth_model_data/flooded_imagery_CB_2023')

Copying files: 100%|██████████| 1217/1217 [00:04<00:00, 302.47it/s]
